In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Introduction :
* Since there are 50 datasets(in`.txt`format), I hope to create a common way to find the variables which include the number of banks as much as possible and plot these selected variables,using `kernal distribution, histogram, px_scatter as well as seaborn`. These functions can be applied to each schedule of the bank call report dataset.

I have writed these functions and let me introduce each of them :
*  First, `missing_value_display` function is used to calculate the missing values of each column in the dataset, and then sort these variables according to their number of missing values. For example, `In[5]`,the variable `RCON2200 (Total Deposits)` does not have the missing values and is displayed in the last row of this table. The higher you go, the larger the number of missing values。
*  Second,`denplot` function is used to plot the kernal density of the dataset.
*  Third, `histogram_plot` function is used to plot the histogram. Here may be a drawback of this function. For example, when I consider two variables (`RCONB989` and `RCON2200`)in the dataset, `In [5]` tells us that there is 20 missing values in the variable `RCONB989`, so when I plot these two variables at the same time, it means that I have to delete the 20 banks in variable `RCON2200`. I think it may be an underlying drawback, so by default I reccommend to plot four variables at the same time. 
*  Fourth, `choose_variables` function is used to select the variable that we hope to plot. For example `In[8],choose_variables(df_rc,4)[0].head()` is used choosse the top 4 variables which include the dataset as much as possible. It is also possible if you want to plot top 5-8 variables, but here in this function I do not take that into consideration.
*  Fifth, `histogram_n_shares`, I use it to `bar_plot` the top n shares banks in the selected variables. It can also select whether you want to plot(after log transformation or the original data)

## Reminder:
Version 11 is the one with original version

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import gaussian_kde

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 03312020'
df_rc=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 03312020.txt',delimiter="\t")

In [ ]:
def missing_value_display(df):   
    df=df.dropna(how='all',axis=1)
    df.missing=df.isna().sum()
    df.missing=pd.DataFrame(df.missing)
    df.missing.columns=['missing values']
    df.missing['Variable Name']=df.loc[0]
    df_return=df.missing.sort_values(by='missing values',ascending=False)
    return df_return.drop('IDRSSD')

In [ ]:
missing_value_display(df_rc).tail(10)['missing values']

If you want to plot the middle line, such as line 4-8, then use the function `loc()` rather than `tail()` and it can work.

In [ ]:
def denplot(samples,color):
    # plot density curve 
    kde = gaussian_kde(samples.dropna())
    x = np.linspace(np.min(samples),np.max(samples),100)
    plt.plot(x,kde(x),color=color ,linewidth=1.5)
    plt.fill_between(x,0,kde(x),color=color,alpha=0.1)
    plt.plot(samples, np.zeros(samples.shape), color=color ,alpha=0.5,marker='|',markersize=20)
    
def histogram_plot(df):
    colors = ['red','blue','green','orange','purple','pink','brown','cyan','olive','gray']
    k = 0
    fig,axes = plt.subplots(2,2,figsize=(15,12))
    for i in range(2):
        for j in range(2):
            plt.subplot(axes[i,j])
            #接下来这两行是用在edited version里面的：为了将figure中x坐标轴给scale
            #用所有的值除以最大值来scale成(0,1)之间的x轴坐标
            df_plot=df.copy()
            df_plot.iloc[:,k]=df.iloc[:,k]/(max(df.iloc[:,k]))
            df_plot.iloc[:,k].hist(color=colors[k],alpha=0.5,rwidth=0.95,grid=False,density=True,ax=axes[i,j])
            denplot(df_plot.iloc[:,k],colors[k])
            plt.title(df.columns[k])
            k += 1
    plt.legend()
    plt.show()

In [ ]:
def choose_variables(df,n):
    row,col=np.shape(df)
    df_rc=df;
    df_plot=df_rc[missing_value_display(df_rc).tail(n).index].drop(index=0).astype(float)
    df_plot.columns=missing_value_display(df_rc).tail(n)['Variable Name']
    df_plot_log=df_plot[df_plot>n].apply(np.log)
    
    df_plot_log=df_plot_log.dropna(how='any')
    df_plot=df_plot.dropna(how='any')
    miss_values=(missing_value_display(df_rc).tail(n)['missing values']/row)*100
    miss_values=pd.DataFrame(miss_values)
    miss_values['Variable Name']=missing_value_display(df_rc).tail(n)['Variable Name']
    return df_plot,df_plot_log,miss_values

In [ ]:
def histogram_n_shares(df,n):
    colors = ['red','blue','green','orange','purple','pink','brown','cyan','olive','gray']
    k = 0
    fig,axes = plt.subplots(2,2,figsize=(15,12))
    for i in range(2):
        for j in range(2):
            plt.subplot(axes[i,j])
            df_plot=pd.DataFrame(sorted(df.iloc[:,k],reverse=True))
            df_plot=df_plot.dropna()
            df_plot=pd.DataFrame(sorted(df_plot[0],reverse=True))
            df_top_n=np.cumsum(df_plot)/df_plot.sum()
            df_top_n.head(n).plot(kind='bar',color=colors[k],alpha=0.5,grid=False,ax=axes[i,j])
            plt.title(df.columns[k])
            k += 1
    plt.legend()
    plt.show()

In [ ]:
choose_variables(df_rc,4)[0].head()

In [ ]:
choose_variables(df_rc,10)[0].head()

In [ ]:
histogram_plot(choose_variables(df_rc,4)[0])

In [ ]:
histogram_plot(choose_variables(df_rc,4)[1])

In [ ]:
fig = plt.figure(figsize=(16,6))
sns.set(rc={"figure.figsize": (20, 6)}); 
sns.set(style="darkgrid",palette='deep',color_codes=False)
sns.pairplot(choose_variables(df_rc,4)[1], size=4, diag_kind="auto")

In [ ]:
#px.scatter(choose_variables(df_rc,4)[0],x='NON-INTEREST BEARING DEPOSITS',y='TOTAL DEPOSITS',
          #size='TOT INT BEAR DEPS IN DOM&FGN OFF',width=800,trendline="ols")

## Apply these functions to another dataset :

In [ ]:
df_ci_loans=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 03312020.txt',delimiter="\t")

In [ ]:
missing_value_display(df_ci_loans).tail(10)

In [ ]:
choose_variables(df_ci_loans,4)[0].head()

In [ ]:
 histogram_n_shares(choose_variables(df_ci_loans,4)[0],10)

In [ ]:
histogram_plot(choose_variables(df_ci_loans,4)[0])

In [ ]:
histogram_plot(choose_variables(df_ci_loans,4)[1])

In [ ]:
sns.pairplot(choose_variables(df_ci_loans,4)[1], size=4, diag_kind="auto")

In [ ]:
choose_variables(df_ci_loans,4)[0]

In [ ]:
#px.scatter(choose_variables(df_ci_loans,4)[0],x='LN SECURED BY OWNER OCCPD NONFARM NO',y='LN SECURED BY OTHR NONFARM NONRES PR',
          #size='ALL OTH LNS SECD BY 1-4FMLY-JR LIENS',width=800,trendline="ols")

## Empirical Results :

#### Data Loading :

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 03312020'
df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 03312020.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 03312020.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 03312020.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 03312020.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 03312020.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 03312020.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 03312020.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCRI 03312020.txt',delimiter="\t")

For each Schedule, I find the variables with least missing values.

### Missing Rank:

In [ ]:
def missing_rank(lists):
    results=[]
    k=0
    for i in lists:
        temp=choose_variables(i,10)[0].head()
        k+=1
        results.append(temp)

    index_name=[]
    for i in range(7):
        temp=results[i].columns
        index_name.append(temp)
    
    lists_name=['income_statement','balance_sheet','loans_and_lease','tradingassets_and_liability',
       'deposit_liabilities','other_asset','other_liabilities']#,'regulatory_capital']
    df_index_name=pd.DataFrame(index_name).T
    df_index_name.columns=lists_name
    df_index_name
    
    return(df_index_name)

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities,df_regulatory_capital]
missing_rank(lists)

In [ ]:
#lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
 #      df_deposit_liabilities,df_other_asset,df_other_liabilities,df_regulatory_capital]
#results=[]
#for i in lists:
 #   temp=choose_variables(i,10)[0].head()
  #  results.append(temp)

#index_name=[]
#for i in range(8):
    #temp=results[i].columns
    #index_name.append(temp)

In [ ]:
#lists_name=['income_statement','balance_sheet','loans_and_lease','tradingassets_and_liability',
       #'deposit_liabilities','other_asset','other_liabilities','regulatory_capital']
#df_index_name=pd.DataFrame(index_name).T
#df_index_name.columns=lists_name
#df_index_name

### Missing Percentage:

In [ ]:
def percentage_display(lists):
    results=[]
    for i in lists[0:7]:
        temp=choose_variables(i,10)[2]['missing values'].values
        results.append(temp)

    miss_percentage=pd.DataFrame(results).T
    lists_name=['income_statement','balance_sheet','loans_and_lease','tradingassets_and_liability',
       'deposit_liabilities','other_asset','other_liabilities']
    miss_percentage.columns=lists_name
    miss_percentage
    
    return miss_percentage

In [ ]:
percentage_display(lists)

Then given the table above, I choose some important variables, they are:

1. `NET INCOME` from Schedule Income Statement
1. `TOTAL DEPOSITS` from Schedule Balance Sheet
1. `TOTAL TRADING ASSETS` and `TRADING LIABILITES` from Schedule Trading Asset and Liability
1. `OTHER EXPENSES ACCRUED & UNPAID` from Schedule Other Liabilities
1. `COMMON EQUITY TIER 1 CAPITAL` from Schedule Regulatory Asset

In [ ]:
for i in lists:
    i.columns=i.iloc[0,:]
    i.drop([0],axis=0,inplace=True)

In [ ]:
net_income=df_income_statement['NET INCOME'].astype(float)
total_deposits=df_balance_sheet['TOTAL DEPOSITS'].iloc[:,1].astype(float)
total_trading_assets=df_tradingassets_and_liability['TOTAL TRADING ASSETS'].iloc[:,1].astype(float)

total_trading_liabilities=df_tradingassets_and_liability['TRADING LIABILITES'].iloc[:,1].astype(float)
other_expenses=df_other_liabilities['OTHER EXPENSES ACCRUED & UNPAID'].iloc[:,1].astype(float)
equity_tier1_capital=df_regulatory_capital['COMMON EQUITY TIER 1 CAPITAL'].iloc[:,1].astype(float)


In [ ]:
df_new=[net_income,total_deposits,total_trading_assets,total_trading_liabilities,other_expenses,equity_tier1_capital]
df_new_plot=pd.DataFrame(df_new).T
np.shape(df_new_plot)

In [ ]:
np.shape(df_new_plot.dropna(how='any'))

In [ ]:
df_new_plot.describe()

In [ ]:
def histogram_plot(df):
    colors = ['red','blue','green','orange','purple','pink','brown','cyan','olive','gray']
    k = 0
    fig,axes = plt.subplots(2,3,figsize=(20,12))
    lists=[]
    for i in range(2):
        for j in range(3):
            plt.subplot(axes[i,j])
            df.iloc[:,k]=np.where(df.iloc[:,k]<-1e9,np.nan,df.iloc[:,k])
            df.iloc[:,k]=np.where(df.iloc[:,k]>1e9,np.nan,df.iloc[:,k])
            lists.append(df.iloc[:,k].dropna())
            #接下来这两行是用在edited version里面的：为了将figure中x坐标轴给scale
            #用所有的值除以最大值来scale成(0,1)之间的x轴坐标
            lists[k]=lists[k]/(max(lists[k]))
            lists[k].hist(color=colors[k],alpha=0.5,rwidth=0.95,grid=False,density=True,ax=axes[i,j])
            denplot(lists[k],colors[k])
            plt.title(df.columns[k])
            k += 1
    plt.legend()
    plt.show()

In [ ]:
df_new_plot_log=np.where(df_new_plot.apply(np.log)<-1e9,np.nan,df_new_plot.apply(np.log))
df_new_plot_log=pd.DataFrame(df_new_plot_log)
df_new_plot_log.columns=df_new_plot.columns
histogram_plot(df_new_plot_log)

In [ ]:
def histogram_n_shares(df,n):
    colors = ['red','blue','green','orange','purple','pink','brown','cyan','olive','gray']
    k = 0
    fig,axes = plt.subplots(2,3,figsize=(20,12))
    for i in range(2):
        for j in range(3):
            plt.subplot(axes[i,j])
            df_plot=pd.DataFrame(sorted(df.iloc[:,k],reverse=True))
            df_plot=df_plot.dropna()
            df_plot=pd.DataFrame(sorted(df_plot[0],reverse=True))
            df_top_n=np.cumsum(df_plot)/df_plot.sum()
            df_top_n.head(n).plot(kind='bar',color=colors[k],alpha=0.5,grid=False,ax=axes[i,j])
            plt.title(df.columns[k])
            k += 1
    plt.legend()
    plt.show()

In [ ]:
histogram_n_shares(df_new_plot,10)

In [ ]:
def top_n_bank_names(df,n):
    row,col=np.shape(df)
    names=[]
    for i in range(1,col):
        names.append(df.sort_values(by=df.columns[i],ascending=False)['bank id'].head(n).values)
    names_df=pd.DataFrame(names).T
    names_df.columns=df.columns[1:7]
    names_df=names_df.astype(int)
    return names_df
    
    

In [ ]:
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCRI 03312020.txt',delimiter="\t")
bank_id=df_regulatory_capital['IDRSSD'].dropna().astype(float)

df_with_id=df_new_plot.copy()
df_with_id.insert(0,"bank id",bank_id)

top_n_bank_names(df_with_id,10)

In [ ]:
histogram_plot(df_new_plot.dropna(how='any'))

In [ ]:
df_new_plot.head()

In [ ]:
fig = plt.figure(figsize=(16,6))
sns.set(rc={"figure.figsize": (20, 6)}); 
sns.set(style="darkgrid",palette='deep',color_codes=False)
sns.pairplot(df_new_plot[['NET INCOME','TOTAL DEPOSITS','TOTAL TRADING ASSETS',
            'TRADING LIABILITES','COMMON EQUITY TIER 1 CAPITAL']], size=4, diag_kind="auto")

# For the dataset in 2019.03

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 03312019'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 03312019.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 03312019.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 03312019.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 03312019.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 03312019.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 03312019.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 03312019.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCRI 03312019.txt',delimiter="\t")

In [ ]:
#df_regulatory_capital.drop(['RCFA7204','RCFA7205','RCFA7206','ADVANCED APPROACHES ONLY: SUPPLEMENT',
 #                           'CAPTIAL CONSERVATION BUFFER','CAP RATIO COM EQTY TIER1 CAP RATIO','TIER 1 LEVERAGE CAPITAL RATIO',
  #                         'TOTAL RISK-BASED CAPITAL RATIO','TIER 1 RISK-BASED CAPITAL RATIO','CAPTIAL CONSERVATION BUFFER',
   #                        'CAP RATIO COM EQTY TIER1 CAP RATIO'],axis=1,inplace=True)

In [ ]:
def missing_rank(lists):
    results=[]
    k=0
    for i in lists:
        temp=choose_variables(i,10)[0].head()
        k+=1
        results.append(temp)

    index_name=[]
    for i in range(7):
        temp=results[i].columns
        index_name.append(temp)
    
    lists_name=['income_statement','balance_sheet','loans_and_lease','tradingassets_and_liability',
       'deposit_liabilities','other_asset','other_liabilities']#,'regulatory_capital']
    df_index_name=pd.DataFrame(index_name).T
    df_index_name.columns=lists_name
    df_index_name
    
    return(df_index_name)

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)

### As for total asset:

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

# For the dataset in 2018.06

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 06302018'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 06302018.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 06302018.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 06302018.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 06302018.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 06302018.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 06302018.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 06302018.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCRI 06302018.txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

# For the dataset in 2017.12

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312017'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312017.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312017.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312017.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312017.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312017.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312017.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312017.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCRI 12312017.txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

# For the dataset in 2013.12

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312013'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312013.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312013.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312013.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312013.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312013.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312013.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312013.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCR 12312013(1 of 2).txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

# For the dataset in 2008.12

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312008'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312008.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312008.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312008.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312008.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312008.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312008.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312008.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCR 12312008(1 of 2).txt',delimiter="\t")

In [ ]:
#path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312010'

#df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312010.txt',delimiter="\t")
#df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312010.txt',delimiter="\t")
#df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312010.txt',delimiter="\t")
#df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312010.txt',delimiter="\t")

#df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312010.txt',delimiter="\t")
#df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312010.txt',delimiter="\t")
#df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312010.txt',delimiter="\t")
#df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCR 12312010(1 of 2).txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

# For the dataset in 2007.12

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312007'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312007.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312007.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312007.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312007.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312007.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312007.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312007.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCR 12312007(1 of 2).txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

In [ ]:
for i in lists:
    i.columns=i.iloc[0,:]
    i.drop([0],axis=0,inplace=True)

df_regulatory_capital.columns=df_regulatory_capital.iloc[0,:]
df_regulatory_capital.drop([0],axis=0,inplace=True)

In [ ]:

leverage_equity_asset=df_regulatory_capital['TOTAL EQUITY CAPITAL'].iloc[:,1].astype(float)/df_balance_sheet['TOTAL ASSETS'].iloc[:,1].astype(float)
leverage_equity_asset=leverage_equity_asset.astype(float)
leverage_equity_asset=pd.DataFrame(leverage_equity_asset)
leverage_equity_asset.columns=['Leverage Equity Asset']

leverage_liability_asset=df_tradingassets_and_liability['TRADING LIABILITES'].iloc[:,1].astype(float)/df_balance_sheet['TOTAL ASSETS'].iloc[:,1].astype(float)
leverage_liability_asset=leverage_liability_asset.astype(float)
leverage_liability_asset=pd.DataFrame(leverage_liability_asset)
leverage_liability_asset.columns=['Leverage Equity Asset']


In [ ]:
colors = ['olive','gray']
fig,axes=plt.subplots(1,2,figsize=(15,6))

#plt.subplot(axes[0])
leverage_equity_asset.iloc[:,0].hist(color=colors[0],alpha=0.5,rwidth=0.95,grid=False,density=True,ax=axes[0])
leverage_liability_asset.iloc[:,0].hist(color=colors[1],alpha=0.5,rwidth=0.95,grid=False,density=True,ax=axes[1])

In [ ]:
net_income=df_income_statement['NET INCOME'].astype(float)
total_deposits=df_balance_sheet['TOTAL DEPOSITS'].iloc[:,1].astype(float)
total_trading_assets=df_tradingassets_and_liability['TOTAL TRADING ASSETS'].iloc[:,1].astype(float)

total_trading_liabilities=df_tradingassets_and_liability['TRADING LIABILITES'].iloc[:,1].astype(float)
total_assets=df_balance_sheet['TOTAL ASSETS'].iloc[:,1].astype(float)
total_equity_capital=df_regulatory_capital['TOTAL EQUITY CAPITAL'].iloc[:,1].astype(float)

df_new=[net_income,total_deposits,total_trading_assets,total_trading_liabilities,total_assets,total_equity_capital]
df_new_plot=pd.DataFrame(df_new).T

df_new_plot_log=np.where(df_new_plot.apply(np.log)<-1e9,np.nan,df_new_plot.apply(np.log))
df_new_plot_log=pd.DataFrame(df_new_plot_log)
df_new_plot_log.columns=df_new_plot.columns

In [ ]:
histogram_n_shares(df_new_plot,10)

In [ ]:
histogram_plot(df_new_plot)

In [ ]:
histogram_plot(df_new_plot_log)

In [ ]:
fig = plt.figure(figsize=(16,6))
sns.set(rc={"figure.figsize": (20, 6)}); 
sns.set(style="white",palette='deep',color_codes=False)
sns.pairplot(df_new_plot, size=4,kind='reg', diag_kind="auto")

# For the dataset in 2005.12

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312005'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312005.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312005.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312005.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312005.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312005.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312005.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312005.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCR 12312005(1 of 2).txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()

# For the dataset in 2003.12

In [ ]:
path = '../input/summer-research/FFIEC CDR Call Bulk All Schedules 12312003'

df_income_statement=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RI 12312003.txt',delimiter="\t")
df_balance_sheet=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RC 12312003.txt',delimiter="\t")
df_loans_and_lease=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCCI 12312003.txt',delimiter="\t")
df_tradingassets_and_liability=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCD 12312003.txt',delimiter="\t")

df_deposit_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCE 12312003.txt',delimiter="\t")
df_other_asset=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCF 12312003.txt',delimiter="\t")
df_other_liabilities=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCG 12312003.txt',delimiter="\t")
df_regulatory_capital=pd.read_csv(f'{path}/FFIEC CDR Call Schedule RCR 12312003(1 of 2).txt',delimiter="\t")

In [ ]:
lists=[df_income_statement,df_balance_sheet,df_loans_and_lease,df_tradingassets_and_liability,
       df_deposit_liabilities,df_other_asset,df_other_liabilities]
missing_rank(lists)

In [ ]:
percentage_display(lists)

In [ ]:
df_balance_sheet['RCFD2170'].dropna()